In [8]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from pvlib.solarposition import get_solarposition

In [2]:
df = pd.read_csv("../dataset.csv")
df.head()

,image,bbox,cx,cy,w,h,class_name,image_id,class_id,SL_P1_x,SL_P1_y,SL_P2_x,SL_P2_y,lat,long,height,time,points
0,shanghai_view2_100.png,0.80875 0.655 0.1025 0.405,0.80875,0.65500,0.1025,0.4050,building,0,0,0.68000,0.44375,0.68000,0.42375,31.332769,121.378553,6,2017-06-06-7,"((303, 181), (344, 343))"
1,shanghai_view2_100.png,0.71375 0.58375 0.1025 0.1925,0.71375,0.58375,0.1025,0.1925,building,0,0,0.63750,0.57000,0.63375,0.55000,31.332769,121.378553,6,2017-06-06-7,"((265, 195), (306, 272))"
2,shanghai_view2_100.png,0.49 0.60875 0.105 0.0725,0.49000,0.60875,0.1050,0.0725,building,0,0,0.41250,0.53375,0.40875,0.51500,31.332769,121.378553,33,2017-06-06-7,"((175, 229), (217, 258))"
3,shanghai_view2_100.png,0.37625 0.6375 0.0925 0.09,0.37625,0.63750,0.0925,0.0900,building,0,0,0.33000,0.56000,0.32375,0.54500,31.332769,121.378553,33,2017-06-06-7,"((132, 236), (169, 273))"
4,shanghai_view2_100.png,0.2775 0.675 0.1 0.06,0.27750,0.67500,0.1000,0.0600,building,0,0,0.22375,0.59875,0.21750,0.58500,31.332769,121.378553,6,2017-06-06-7,"((91, 258), (131, 282))"


In [4]:
p1 = df[["SL_P1_x", "SL_P1_y"]].values
p2 = df[["SL_P2_x", "SL_P2_y"]].values

shd_len = np.linalg.norm(p1 - p2, axis=1) # ye square root leta hay
df["shd_len"] = shd_len

In [14]:
def pred_height(row):
    shd_len = row["shd_len"] * 1000
    yr, month, date, hr = row["time"].split("-")
    yr, month, date, hr = int(yr), int(month), int(date), int(hr)
    dt = datetime(yr, month, date, hour=hr)
    lat, long = row["lat"], row["long"]
    solar_angle = np.deg2rad(get_solarposition(dt, lat, long).elevation.values[0])
    height = shd_len / np.tan(solar_angle)
    return height   


In [15]:
df["pred_height"] = df.apply(pred_height, axis=1)

In [16]:
df.head()

,image,bbox,cx,cy,w,h,class_name,image_id,class_id,SL_P1_x,SL_P1_y,SL_P2_x,SL_P2_y,lat,long,height,time,points,shd_len,pred_height
0,shanghai_view2_100.png,0.80875 0.655 0.1025 0.405,0.80875,0.65500,0.1025,0.4050,building,0,0,0.68000,0.44375,0.68000,0.42375,31.332769,121.378553,6,2017-06-06-7,"((303, 181), (344, 343))",0.020000,18.131250
1,shanghai_view2_100.png,0.71375 0.58375 0.1025 0.1925,0.71375,0.58375,0.1025,0.1925,building,0,0,0.63750,0.57000,0.63375,0.55000,31.332769,121.378553,6,2017-06-06-7,"((265, 195), (306, 272))",0.020349,18.447211
2,shanghai_view2_100.png,0.49 0.60875 0.105 0.0725,0.49000,0.60875,0.1050,0.0725,building,0,0,0.41250,0.53375,0.40875,0.51500,31.332769,121.378553,33,2017-06-06-7,"((175, 229), (217, 258))",0.019121,17.334675
3,shanghai_view2_100.png,0.37625 0.6375 0.0925 0.09,0.37625,0.63750,0.0925,0.0900,building,0,0,0.33000,0.56000,0.32375,0.54500,31.332769,121.378553,33,2017-06-06-7,"((132, 236), (169, 273))",0.016250,14.731641
4,shanghai_view2_100.png,0.2775 0.675 0.1 0.06,0.27750,0.67500,0.1000,0.0600,building,0,0,0.22375,0.59875,0.21750,0.58500,31.332769,121.378553,6,2017-06-06-7,"((91, 258), (131, 282))",0.015104,13.692546
